In [2]:
!pip install arcgis
!pip install ipython
!pip install ipywidgets  

## Setup supporting packages

In [3]:
from IPython.display import display, HTML

from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from arcgis.geocoding import geocode

import requests

## HUC12 of interest

In [4]:
#huc12 = '051302030106' # Tennessee
#huc12 = '020700100204' #DC
#huc12 = '020700100103' #DC and VA
#huc12 = '031501060703' #Alpine, al
huc12 = '14080101' #Upper San Juan

# Query ArcGIS based watershed web service

In [5]:
huc_result = None

huc_ags_sevice_url = 'https://inlandwaters.geoplatform.gov/arcgis/rest/services/NHDPlus/WatershedBoundaryDataset/MapServer/10'
huc12_conus = FeatureLayer(huc_ags_sevice_url)

huc4 = huc12[0:4]
print("HUC4 = " + huc4)

huc_result = huc12_conus.query(where="HUC12 like '" + huc4 + "%'", 
                                    out_fields='HUC12,Name,AREAACRES,AREASQKM,STATES',return_geometry=False)

if huc_result == None or huc_result.sdf.empty:
    print("\nUnable to locate HUC12 information for " + huc12)
else:    
    display(huc_result.sdf)

HUC4 = 1408


,AREAACRES,AREASQKM,HUC12,NAME,OBJECTID,STATES
0,23717.752619,95.982329,140802041402,Lower Church Rock Valley,22218,AZ
1,10784.798261,43.644526,140801060701,Upper De-na-zin Wash,22220,NM
2,11466.481162,46.403198,140802041303,Long House Valley,22509,AZ
3,13821.481840,55.933547,140802041304,Wildcat Canyon-Laguna Creek,22511,AZ
4,29264.897754,118.430827,140802041306,Marsh Pass-Laguna Creek,22513,AZ
5,29636.845531,119.936046,140802041401,Upper Church Rock Valley,22514,AZ
6,22638.112403,91.613181,140802041307,Town of Kayenta-Laguna Creek,22515,AZ
7,37531.293903,151.883742,140802040602,Upper Cottonwood Wash,22518,AZ
8,16223.986956,65.656139,140802040601,Black Mountain Trading Post Area,22521,AZ
9,27823.704237,112.598524,140802040708,Nazlini Wash (Local Drainage),22574,AZ


## Build a list of unique ATTAINS assestment units

In [24]:
def print_nested(val, nesting = -5):
    if type(val) == dict:
        print('')
        nesting += 5
        for k in val:
            print(nesting * ' ', end='')
            print(k, end=':')
            print_nested(val[k],nesting)
    else:
        print(val)

len(huc_result.to_dict())

4

In [30]:
len(huc_result.to_dict()["features"])

713

In [31]:
print(huc_result)

{"features": [{"attributes": {"OBJECTID": 22218, "HUC12": "140802041402", "NAME": "Lower Church Rock Valley", "AREAACRES": 23717.7526190172, "AREASQKM": 95.9823294927854, "STATES": "AZ"}}, {"attributes": {"OBJECTID": 22220, "HUC12": "140801060701", "NAME": "Upper De-na-zin Wash", "AREAACRES": 10784.7982605063, "AREASQKM": 43.644525549319404, "STATES": "NM"}}, {"attributes": {"OBJECTID": 22509, "HUC12": "140802041303", "NAME": "Long House Valley", "AREAACRES": 11466.481161675, "AREASQKM": 46.4031980880116, "STATES": "AZ"}}, {"attributes": {"OBJECTID": 22511, "HUC12": "140802041304", "NAME": "Wildcat Canyon-Laguna Creek", "AREAACRES": 13821.481839742799, "AREASQKM": 55.933546712054, "STATES": "AZ"}}, {"attributes": {"OBJECTID": 22513, "HUC12": "140802041306", "NAME": "Marsh Pass-Laguna Creek", "AREAACRES": 29264.8977541975, "AREASQKM": 118.43082706596101, "STATES": "AZ"}}, {"attributes": {"OBJECTID": 22514, "HUC12": "140802041401", "NAME": "Upper Church Rock Valley", "AREAACRES": 29636.8

In [42]:
for feature in huc_result.to_dict()["features"]:
    huc12_value = feature["attributes"]["HUC12"]
    print(huc12_value)

140802041402
140801060701
140802041303
140802041304
140802041306
140802041401
140802041307
140802040602
140802040601
140802040708
140802040604
140802040707
140802040606
140802040607
140802040801
140802040605
140802040805
140802040808
140802040807
140802040806
140802040809
140802040802
140802040804
140802040803
140802041502
140802041205
140802041206
140802041201
140802041202
140802041210
140802041209
140802041204
140802041203
140802041207
140802041101
140802041102
140802041104
140802041103
140802041501
140802041105
140802040905
140802041011
140802041503
140802041504
140802041012
140802041208
140802040501
140802040701
140801060901
140801060903
140801061002
140801060904
140801061201
140801060902
140801061006
140801061101
140801061004
140801061001
140801061003
140801061302
140801061005
140801061202
140801061102
140801061301
140801061103
140801061203
140801060405
140801060802
140801060803
140801060404
140801060408
140801060801
140801060502
140801060501
140801060603
140801060804
140801060805

In [46]:
url = 'https://attains.epa.gov/attains-public/api/huc12summary?huc=' + huc12_value
print("\nWeb service url = " + url)
response = requests.get(url)        # To execute get request 
#print(response.status_code)     # To print http response code  
#print(response.text)            # To print formatted JSON response 

data = response.json()

assessmentUnits = data['items'][0]['assessmentUnits']

assessmentUnitList = []
for unit in assessmentUnits:
    assessmentUnitList.append(unit['assessmentUnitId'])

assessmentUnitList = ",".join(map(lambda x: "'" + str(x) + "'",assessmentUnitList))

if assessmentUnitList == "":
    print("\nNo assessment units for HUC = " + huc12)
else:
    print("\nThe following asseessment untis where found for HUC = " + huc12 + "\n")
    print(assessmentUnitList + "\n")


Web service url = https://attains.epa.gov/attains-public/api/huc12summary?huc=140802040605

No assessment units for HUC = 14080101
